### Water TIP4P/Ice, N = 1000

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import fft
import array as arr
import statistics
import seaborn as sns
import scipy as scipy
import scipy.fft
from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy import stats
from pylab import *
import matplotlib.pyplot as plt
from scipy import signal
import k3d
from k3d.colormaps import matplotlib_color_maps
from k3d.helpers import map_colors

In [ ]:
T=190 #K
# Conversion factors and constants:
nm_m=1e-9 # conversion factor from nm to m
eAng_Cm=1.60217662e-19*1e-10 # conversion factor from electron charge*Angstrom to Coulomb*m
D_Cm=3.33584e-30 # conversion factor from Debye to C*m
eAng_D=eAng_Cm/D_Cm # conversion factor from charge*Angstrom to Debye
kB=8.61733326e-5 # [eV/K] Boltzmann constant
kB_SI=1.38e-23 # [J/K] Boltzmann constant
e_0SI=8.854e-12 # [F/m=C^2/Jm] vacuum permittivity
J_eV=6.242e18 #conversion factor J to eV
e_0=e_0SI/D_Cm**2/J_eV # [D^2/eVm^3] vacuum permittivity
atm_Pa=1e5 # conversion factor from atm to Pa
m_h2o = 18.01528 #[g/mol]
mol=6.02214e23 # mole
eAng_Cm=1.60217662e-19*1e-10 # conversion factor from electron charge*Angstrom to Coulomb*m
D_Cm=3.33584e-30 # conversion factor from Debye to C*m
dipTIP4Pice=2.426*D_Cm/eAng_Cm/10 #single-molecule dipole (e*nm) of TIP4Pice water 

In [ ]:
fileDensity='density.dat' #import density time series data

In [ ]:
dataDensity = pd.read_csv(fileDensity,sep='\s+').values

In [ ]:
%matplotlib notebook
plt.figure(figsize=(16,5))
plt.plot(dataDensity[:,0],dataDensity[:,1],'--')
plt.xlabel("t (ps)", fontsize=18)
plt.ylabel("Density ($kg/m^3$)", fontsize=18)
plt.show()

In [ ]:
fileEnergy='energy.dat' #import energy time series data

In [ ]:
dataEnergy = pd.read_csv(fileEnergy,sep='\s+').values

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(dataEnergy[:,0],dataEnergy[:,1],'--')
plt.xlabel("t (ps)", fontsize=18)
plt.ylabel("Energy (kJ/mol)", fontsize=18)
plt.show()

In [ ]:
filePressure='pressure.dat' #import pressure time series data

In [ ]:
dataPressure = pd.read_csv(filePressure,sep='\s+').values

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(dataPressure[:,0],dataPressure[:,1],'--')
plt.xlabel("t (ps)", fontsize=18)
plt.ylabel("P(bar)", fontsize=18)
plt.show()

In [ ]:
fileVolume='volume.dat' #import volume time series data

In [ ]:
dataVolume = pd.read_csv(fileVolume,sep='\s+').values

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(dataVolume[:,0],dataVolume[:,1],'--')
plt.xlabel("t (ps)", fontsize=18)
plt.ylabel("V ($nm^3$)", fontsize=18)
plt.show()

In [ ]:
charge=np.zeros((4000, 1))
nMass=np.zeros((4000, 1))

In [ ]:
#TIP4P/Ice: q(O) = 0; q(H) = 0.5897 e; q(O) = -2q(H) = -1.1794 e
for n in range(0,1000):
    charge[0+4*n,0]=0
    charge[1+4*n,0]=0.5897
    charge[2+4*n,0]=0.5897
    charge[3+4*n,0]=-1.1794 
    nMass[0+4*n,0]=15.99940 #amu
    nMass[1+4*n,0]=1.00794 #amu
    nMass[2+4*n,0]=1.00794 #amu
    nMass[3+4*n,0]=0 

In [ ]:
totConf = 500  # Total number of configurations (simulation time steps)

In [ ]:
M = np.zeros((totConf,5))
# M[:,0]: simulation time step (i.e., configuration index) 
# M[:,i], i=1,2,3: total polarization vector components along the Cartesian axes 
# M[:,4]: total polarization magnitude
Msm=np.zeros((totConf,5,1000))
# n = (0,999) is the molecule index 
# Msm[:,0,n]: simulation time step (i.e., configuration index)
# Msm[:,i,n], i=1,2,3: dipole vector components along the Cartesian axes for the n-th molecule
# Msm[:,4,n]: dipole magnitude for the n-th molecule
CM=np.zeros((totConf,4,1000))
# n = (0,999) is the molecule index 
# CM[:,0,n]: simulation time step (i.e., configuration index)
# CM[:,i,n], i=1,2,3: spatial coordinates of the center of mass of the n-th molecule
CMu=np.zeros((totConf,4,1000))
CMun=np.zeros((totConf,4,1000))
# n = (0,999) is the molecule index 
# CMun[:,0,n]: simulation time step (i.e., configuration index)
# CMun[:,i,n], i=1,2,3: unwrapped spatial coordinates of the center of mass of the n-th molecule
Lbox=np.zeros((totConf,3,1000))
# simulation box size at each simulation time step
coo=np.zeros((totConf,3,4000))
# spatial coordinates at each simulation time step of the 4 particles (two hydrogen atoms, one oxygen atom and M-site particle) composing TIP4P/Ice water molecule for all molecules
chargetotConf=np.zeros((totConf,4000))
# electric charge of the 4 particles (two hydrogen atoms, one oxygen atom and M-site particle) composing TIP4P/Ice water molecule for all molecules
nMasstotConf=np.zeros((totConf,4000))
# atomic mass number of the 4 particles (two hydrogen atoms, one oxygen atom and M-site particle) composing TIP4P/Ice water molecule for all molecules

In [ ]:
for m in range(0,totConf):
    chargetotConf[m,:]=charge[:,0]
    nMasstotConf[m,:]=nMass[:,0]  

In [ ]:
mO=15.99940 #amu
mH=1.00794 #amu
dOc2H = 0.9572 * np.cos(104.52 / 180 / 2 * np.pi)  # angstroms; distance between the oxygen atom and the center of mass of the two hydrogen atoms
dOM = 0.1546  # angstroms; distance between the oxygen atom and the M-site particle
mMcm=mO*(dOc2H-dOM)/dOc2H/(mO+2*mH)

In [ ]:
for i in range(1,totConf+1):
    M[i-1,0]=i*8*10000
    filename='CONFS/CnfT190t' + str(i*8) + '0000'
    globals()['data%s' % str(i*8) + '0000']=pd.read_csv(filename,sep='\s+',header=None,skiprows=2).values 
    L=float(globals()['data%s' % str(i*8) + '0000'][4000,1])
    for l in range(0,1000):
        for n in range(1,4):
        # Unwrap atom coordinates if necessary to compute individual molecule dipoles. 
        # Note: this does not unwrap the center of mass coordinates.
            cooO=globals()['data%s' % str(i*8) + '0000'][4*l,2+n] 
            cooM=globals()['data%s' % str(i*8) + '0000'][4*l+3,2+n]
            if abs(globals()['data%s' % str(i*8) + '0000'][4*l+3,2+n]-globals()['data%s' % str(i*8) + '0000'][4*l+1,2+n])<=0.1:
                cooH1=globals()['data%s' % str(i*8) + '0000'][4*l+1,2+n]
            elif globals()['data%s' % str(i*8) + '0000'][4*l+1,2+n]<globals()['data%s' % str(i*8) + '0000'][4*l+3,2+n]:
                cooH1=globals()['data%s' % str(i*8) + '0000'][4*l+1,2+n]+L
            else:
                cooH1=globals()['data%s' % str(i*8) + '0000'][4*l+1,2+n]-L
            if  abs(globals()['data%s' % str(i*8) + '0000'][4*l+3,2+n]-globals()['data%s' % str(i*8) + '0000'][4*l+2,2+n])<=0.1:
                cooH2=globals()['data%s' % str(i*8) + '0000'][4*l+2,2+n]                
            elif globals()['data%s' % str(i*8) + '0000'][4*l+2,2+n]<globals()['data%s' % str(i*8) + '0000'][4*l+3,2+n]: 
                cooH2=globals()['data%s' % str(i*8) + '0000'][4*l+2,2+n]+L
            else:
                cooH2=globals()['data%s' % str(i*8) + '0000'][4*l+2,2+n]-L
            Msm[i-1,n,l] = cooM*charge[4*l+3,0]+cooH1*charge[4*l+1,0]+cooH2*charge[4*l+2,0]
            CM[i-1,n,l] = (cooM*mMcm+cooH1*mH+cooH2*mH)/(mMcm+mH+mH)
            Lbox[i-1,n-1,l]=L
            # Unwrap center of mass molecules coordinates if necessary
            CMu[0,n,l] = CM[0,n,l]
            CMun[0,n,l] = CM[0,n,l]-L/2
            if i>1:
                CMu[i-1,n,l] = CM[i-1,n,l]-floor((CM[i-1,n,l]-CMu[i-2,n,l])/L+0.5)*L
                CMun[i-1,n,l]=CMu[i-1,n,l]-L/2
            coo[i-1,n-1,4*l] = cooO
            coo[i-1,n-1,4*l+1] = cooH1
            coo[i-1,n-1,4*l+2] = cooH2
            coo[i-1,n-1,4*l+3] = cooM
        Msm[i-1,4,l]=(Msm[i-1,1,l]**2+Msm[i-1,2,l]**2+Msm[i-1,3,l]**2)**0.5  
    M[i-1,1]=sum(Msm[i-1,1,:])
    M[i-1,2]=sum(Msm[i-1,2,:])
    M[i-1,3]=sum(Msm[i-1,3,:])
    M[i-1,4]=(M[i-1,1]**2+M[i-1,2]**2+M[i-1,3]**2)**0.5

### Plot the spatial distribution of molecular dipoles in Cartesian coordinates.

In [ ]:
CMTOT=np.zeros((3,totConf))
for i in range(1,totConf):
    CMTOT[:,i]=sum(CMu[i,1:,:],axis=1)/1000

In [ ]:
c=250 #select configuration

In [ ]:
# Dipoles are represented by arrows here.

# Initialize a 3D plot
plot = k3d.plot(camera_auto_fit=False)

# Set scaling factor for molecular dipole length to improve visualization
s0 = 0.1 

# Loop through each dipole to add them as vectors in the plot
for l in range(0, 999):
    # Define a vector for each dipole based on its position and scaled direction
    line = k3d.vectors(
        [CMu[c, 1, l], CMu[c, 2, l], CMu[c, 3, l]],  # Starting point (position) of the dipole
        [s0 * Msm[c, 1, l] / Msm[c, 4, l],           # Scaled x-component of the dipole vector
         s0 * Msm[c, 2, l] / Msm[c, 4, l],           # Scaled y-component of the dipole vector
         s0 * Msm[c, 3, l] / Msm[c, 4, l]],          # Scaled z-component of the dipole vector
        line_size=0.1,                               
        color_map=k3d.basic_color_maps.Jet           
    )
    plot += line  # Add each vector to the plot

# Define a larger scaling factor for visualizing the total polarization vector
s1 = 10  

# Plot the center of mass of the entire system as a red point
pointCM = k3d.points(
    positions=[CMTOT[0, c], CMTOT[1, c], CMTOT[2, c]],  # Coordinates of the center of mass
    point_size=0.1,                                     
    shader='3d',                                        
    color=0xFF0000                                      
)

# Define a vector for the total polarization at the center of mass
lineCM = k3d.vectors(
    [CMTOT[0, c], CMTOT[1, c], CMTOT[2, c]],             # Starting point at the center of mass
    [s1 * M[c, 1] / M[c, 4],                             # Scaled x-component of polarization vector
     s1 * M[c, 2] / M[c, 4],                             # Scaled y-component of polarization vector
     s1 * M[c, 3] / M[c, 4]],                            # Scaled z-component of polarization vector
    line_size=5,                                         
    color=0xFF0000                                       # Set color to red
)

# Display the 3D plot
plot.display()

In [ ]:
#The color map of the points represents here the projection of each dipole along the direction of the total polarization.
# Initialize arrays for attributes and positions
attr = np.zeros(1000)   # Array to store calculated attributes for each point
x = np.zeros((1000, 3)) # Array to store 3D coordinates for each point

# Loop over each point to extract positions and calculate attributes
for l in range(0, 1000): 
    # Extract 3D coordinates from CM array for point `l`
    for n in range(0, 3):
        x[l, n] = CM[c, n + 1, l]   # Position coordinates

    # Calculate the attribute value for each point `l` based on specific ratios
    attr[l] = (
        (Msm[c, 1, l] / Msm[c, 4, l]) * (M[c, 1] / M[c, 4]) +
        (Msm[c, 2, l] / Msm[c, 4, l]) * (M[c, 2] / M[c, 4]) +
        (Msm[c, 3, l] / Msm[c, 4, l]) * (M[c, 3] / M[c, 4])
    )

# Initialize a 3D plot in k3d
plot2 = k3d.plot()

# Create points for the plot with attributes and a color map
# Set color map to Viridis and map colors to the attribute range
points = k3d.points(
    positions=x,                   # Use positions array for point coordinates
    point_size=0.2,                
    shader='3d',                   
    color_map=matplotlib_color_maps.Viridis,  
    color_range=[attr.min(), attr.max()]      
)
points.attribute = attr  # Assign the attribute values to the points for color mapping

# Add points to the plot and display it
plot2 += points
plot2.display()

In [ ]:
# Plotting the distribution of the projection of each dipole along the direction of the total polarization.
# Create a histogram with KDE (density line)
plt = sns.histplot(attr, bins=100, kde=False, color='grey', stat="density") 
sns.kdeplot(attr, color='black', linewidth=2)  # Add a black KDE line 
#
plt.set_xlabel('$\mathbf{d}_i \cdot \hat{p}$', fontsize=28)  
plt.set_ylabel('Frequency', fontsize=28)  
#
plt.tick_params(axis='both', which='major', labelsize=28, length=10, width=2, color='black')  
plt.tick_params(axis='both', which='minor', length=7, width=1)  
#
plt.minorticks_on()

In [ ]:
Lu = np.max(CMu[c, :, :])  

# Define grid parameters for dividing the spatial domain into cells
n_cell = 3  # Number of cells along each dimension
CM_cell = np.zeros((n_cell, n_cell, n_cell, 3))  # Array to store center of mass for each cell
Msm_cell = np.zeros((n_cell, n_cell, n_cell, 3))  # Array to store polarization vector for each cell
count = np.zeros((n_cell, n_cell, n_cell, 1))  # Counter to track the number of entries per cell

# Loop to assign values in cells
for nx in range(n_cell):
    for ny in range(n_cell):
        for nz in range(n_cell):
            for l in range(1000):
                if (nx * Lu / n_cell <= CMu[c, 1, l] < (nx + 1) * Lu / n_cell and
                    ny * Lu / n_cell <= CMu[c, 2, l] < (ny + 1) * Lu / n_cell and
                    nz * Lu / n_cell <= CMu[c, 3, l] < (nz + 1) * Lu / n_cell):    
                    count[nx, ny, nz] += 1  
                    CM_cell[nx, ny, nz, :] += CMu[c, 1:4, l]  
                    Msm_cell[nx, ny, nz, :] += Msm[c, 1:4, l] 
            # Compute the average values in the cell, adding a small epsilon for stability
            CM_cell[nx, ny, nz, :] /= (count[nx, ny, nz] + 1e-6)

# Reshape CM and Msm matrices into 1D arrays for easy indexing
CM_cell = CM_cell.reshape((n_cell**3, 3))
Msm_cell = Msm_cell.reshape((n_cell**3, 3))

# Apply a mask to filter out zero rows in CM_cell and apply the same to Msm_cell
mask = np.all(CM_cell != 0, axis=1)
CM_cell = CM_cell[mask]
Msm_cell = Msm_cell[mask]

# Calculate the magnitude of each vector in Msm_cell
Msm_cellMod = np.linalg.norm(Msm_cell, axis=1)

In [ ]:
# initialize arrays 
Msm_cellPN = np.zeros_like(Msm_cell)
Msm_cellModPN = np.zeros(Msm_cell.shape[0])

for l in range(Msm_cell.shape[0]): 
    for i in range(3):
        # Compute the projection of MsM_cell onto the plane orthogonal to the total polarization direction.
        Msm_cellPN[l, i] = Msm_cell[l, i] / Msm_cellMod[l] - (Msm_cell[l, i] / Msm_cellMod[l]) * (M[c, i + 1] / M[c, 4])
    
    # Calculate the magnitude of Msm_cell
    Msm_cellModPN[l] = np.linalg.norm(Msm_cellPN[l, :])

# Compute the average of Msm_cell over cells
Msm_cellPNav = np.mean(Msm_cellPN, axis=0)

# Calculate the magnitude of the averaged Msm_cell
Msm_cellPNavMod = np.linalg.norm(Msm_cellPNav)

In [ ]:
subplot
plot1cellN = k3d.plot()  
plot1cellN.display()  

# Initialize attributes and positions
attr = np.zeros(CM_cell[:, 0].size)  # Array to store computed attributes
x = np.zeros((CM_cell[:, 0].size, 3))  # Array to store positions of points

# Iterate over each cell to calculate attributes and positions
for l in range(0,CM_cell[:,0].size): 
    for n in range(0,3):
        x[l,n]=CM_cell[l,n] # Assign position from CM_cell
    attr[l]=(Msm_cell[l,0]/Msm_cellMod[l])*(M[c,1]/M[c,4])+(Msm_cell[l,1]/Msm_cellMod[l])*(M[c,2]/M[c,4])+(Msm_cell[l,2]/Msm_cellMod[l])*(M[c,3]/M[c,4]) # the attribute is the projection of Msm_cell onto the direction of the total polarization vector
points = k3d.points(
    positions=x,
    point_size=0.4,
    shader='3d',
    color_map=matplotlib_color_maps.Viridis,  
    color_range=[attr.min(), attr.max()]  
)
points.attribute=attr
plot1cellN += points

for l in range(0,CM_cell[:,0].size): 
    line = k3d.vectors([CM_cell[l,0],CM_cell[l,1],CM_cell[l,2]],[((Msm_cell[l,0]/Msm_cellMod[l])*(M[c,1]/M[c,4])+(Msm_cell[l,1]/Msm_cellMod[l])*(M[c,2]/M[c,4])+(Msm_cell[l,2]/Msm_cellMod[l])*(M[c,3]/M[c,4]))*(M[c,1]/M[c,4]),((Msm_cell[l,0]/Msm_cellMod[l])*(M[c,1]/M[c,4])+(Msm_cell[l,1]/Msm_cellMod[l])*(M[c,2]/M[c,4])+(Msm_cell[l,2]/Msm_cellMod[l])*(M[c,3]/M[c,4]))*(M[c,2]/M[c,4]),((Msm_cell[l,0]/Msm_cellMod[l])*(M[c,1]/M[c,4])+(Msm_cell[l,1]/Msm_cellMod[l])*(M[c,2]/M[c,4])+(Msm_cell[l,2]/Msm_cellMod[l])*(M[c,3]/M[c,4]))*(M[c,3]/M[c,4])],line_width=0.02,head_size=1) #the arrows represent the projection of Msm_cell onto the direction of the total polarization vector
    plot1cellN += line
    
lineCM = k3d.vectors([CMTOT[0,c], CMTOT[1,c], CMTOT[2,c]], 
                     [M[c,1] / M[c,4], M[c,2] / M[c,4], M[c,3] / M[c,4]], 
                     line_size=5, color=0xFF0000,line_width=0.05,head_size=2)  
pointCM = k3d.points(positions=[CMTOT[0,c], CMTOT[1,c], CMTOT[2,c]], 
                     point_size=0.1, 
                     shader='3d', 
                     color=0xFF0000)  
plot1cellN += pointCM
plot1cellN += lineCM

subplot
plot1cellPN=k3d.plot()
plot1cellPN.display()
attr=np.zeros(CM_cell[:,0].size)
x=np.zeros((CM_cell[:,0].size,3))

for l in range(0,CM_cell[:,0].size): 
    for n in range(0,3):
        x[l,n]=CM_cell[l,n] # Assign position from CM_cell
    attr[l]=((Msm_cell[1,0]/Msm_cellMod[1]-(Msm_cell[1,0]/Msm_cellMod[1])*(M[c,1]/M[c,4]))*(Msm_cell[l,0]/Msm_cellMod[l]-(Msm_cell[l,0]/Msm_cellMod[l])*(M[c,1]/M[c,4]))+(Msm_cell[1,1]/Msm_cellMod[1]-(Msm_cell[1,1]/Msm_cellMod[1])*(M[c,2]/M[c,4]))*(Msm_cell[l,1]/Msm_cellMod[l]-(Msm_cell[l,1]/Msm_cellMod[l])*(M[c,2]/M[c,4]))+(Msm_cell[1,2]/Msm_cellMod[1]-(Msm_cell[1,2]/Msm_cellMod[1])*(M[c,3]/M[c,4]))*(Msm_cell[l,2]/Msm_cellMod[l]-(Msm_cell[l,2]/Msm_cellMod[l])*(M[c,3]/M[c,4])))/Msm_cellModPN[1]/Msm_cellModPN[l] #attribute here is the projection of Msm_cell onto the plane orthogonal to the direction of the total polarization vector

points = k3d.points(
    positions=x,
    point_size=0.4,
    shader='3d',
    color_map=matplotlib_color_maps.Viridis,  
    color_range=[attr.min(), attr.max()]  
)
points.attribute=attr
plot1cellPN += points
for l in range(0,CM_cell[:,0].size): 
    line = k3d.vectors([CM_cell[l,0],CM_cell[l,1],CM_cell[l,2]],[Msm_cell[l,0]/Msm_cellMod[l]-(Msm_cell[l,0]/Msm_cellMod[l])*(M[250,1]/M[250,4]),Msm_cell[l,1]/Msm_cellMod[l]-(Msm_cell[l,1]/Msm_cellMod[l])*(M[250,2]/M[250,4]),Msm_cell[l,2]/Msm_cellMod[l]-(Msm_cell[l,2]/Msm_cellMod[l])*(M[250,3]/M[250,4])],line_size=0.2,line_width=0.02,head_size=1) #arrows represent the projection of Msm_cell onto the plane orthogonal to the total polarization vector.
    plot1cellPN += line
lineCM = k3d.vectors([CMTOT[0,c], CMTOT[1,c], CMTOT[2,c]], 
                     [M[c,1] / M[c,4], M[c,2] / M[c,4], M[c,3] / M[c,4]], 
                     line_size=5, color=0xFF0000,line_width=0.05,head_size=2) 
pointCM = k3d.points(positions=[CMTOT[0,c], CMTOT[1,c], CMTOT[2,c]], 
                     point_size=0.1, 
                     shader='3d', 
                     color=0xFF0000)
plot1cellPN += pointCM
plot1cellPN += lineCM

In [ ]:
#Reduced time series of density, volume and energy where only the value at the same time step where the polarization is computed are selected  
dataDensity0000 = np.zeros((totConf-1,2))
dataVolume0000 = np.zeros((totConf-1,2))
dataEnergy0000 = np.zeros((totConf-1,2))

In [ ]:
for i in range(0,totConf-1): 
    dataDensity0000[i,0]=dataDensity[(i+1)*499+i,0]
    dataDensity0000[i,1]=dataDensity[(i+1)*499+i,1]
    dataVolume0000[i,0]=dataVolume[(i+1)*499+i,0]
    dataVolume0000[i,1]=dataVolume[(i+1)*499+i,1]
    dataEnergy0000[i,0]=dataEnergy[(i+1)*499+i,0]
    dataEnergy0000[i,1]=dataEnergy[(i+1)*499+i,1]

### Compute $\epsilon_0$, $k_T$, $U_L$, $\langle \rho \rangle$.

In [ ]:
V_mean = statistics.mean(dataVolume[:,1])*nm_m**3

In [ ]:
for i in range (1,4):
    M[:,i]=M[:,i]*10*eAng_D

In [ ]:
M[:,4]=M[:,4]*10*eAng_D

#### Dielectric constant ($\epsilon_0$)

In [ ]:
# Total number of timesteps
Nt = len(M[:, 0])                                    

# Define the range of block sizes
minBlockSize = 4                                     # Minimum block size: at least 1 observation per block
maxBlockSize = int(Nt / 4)                           # Maximum block size: ensures at least 4 blocks for variance computation
NumBlocks = maxBlockSize - minBlockSize              # Total number of block sizes 

# Initialize arrays to store results
blockMeanVarM = np.zeros([3, NumBlocks])             
blockVarVarM = np.zeros([3, NumBlocks])              
blockCtr = 0                                         # Counter for block sizes

# Iterate through block sizes
for blockSize in range(minBlockSize, maxBlockSize):
    Nblock = int(Nt / blockSize)                     
    M_varblock = np.zeros([3, Nblock])               
    

    for i in range(1, Nblock + 1):                  
        ibeg = (i-1)*blockSize                       # Start index of the block
        iend = ibeg+blockSize                        # End index of the block
        
        # Calculate variance in the block for each polarization component
        for n in range(1, 4):                       
            M_varblock[n - 1, i - 1] = statistics.variance(M[ibeg:iend, n])
    
    # Calculate mean and variance of block polarization variances
    for m in range(1, 4):                           
        blockMeanVarM[m - 1, blockCtr] = statistics.mean(M_varblock[m - 1, :])
        blockVarVarM[m - 1, blockCtr] = statistics.variance(M_varblock[m - 1, :]) / (Nblock - 1)
    
    # Increment block size counter
    blockCtr += 1

In [ ]:
e0_block=1+sum(blockMeanVarM[:,:],axis=0)/(3*e_0*V_mean*kB*T)
de0_block=np.sqrt(blockVarVarM[0,:]+blockVarVarM[1,:]+blockVarVarM[2,:])/(3*e_0*V_mean*kB*T)

In [ ]:
e0 = statistics.mean(e0_block[#x:#y]) #fill with the proper x,y
de0 = statistics.mean(de0_block[#x:#y]) #fill with the proper x,y
print('Static dielectric permittivity e0=', e0, '+/-', de0)

#### Isothermal compressibility ($k_T$)

In [ ]:
# Total number of timesteps
Nt= len(dataVolume[:,0])                           

# Initialize arrays to store results
blockMeanVarV = np.zeros([1,NumBlocks])            
blockVarVarV  = np.zeros([1,NumBlocks])            
blockCtr   = 0                             # Counter for block sizes

# Iterate through block sizes
for blockSize in range(minBlockSize, maxBlockSize):
    Nblock    = int(Nt/blockSize)                  
    V_varblock    = np.zeros([1,Nblock])  
    
    for i in range(1,Nblock+1):                    
        ibeg = (i-1) * blockSize
        iend =  ibeg + blockSize
        # Calculate variance in the block for volume
        V_varblock[0,i-1] = statistics.variance(nm_m**3*dataVolume[ibeg:iend,n])
    # Calculate mean and variance of block polarization variances
    blockMeanVarV[0,blockCtr] = statistics.mean(V_varblock[m-1,:])
    blockVarVarV[0,blockCtr]  = statistics.variance(V_varblock[m-1,:])/(Nblock - 1)
    blockCtr += 1

In [ ]:
kT_block=(blockMeanVarV[0,:])/(V_mean*kB_SI*T) # Pa^-1 m^3/J
dkT_block=np.sqrt(blockVarVarV[0,:])/(V_mean*kB_SI*T) # Pa^-1 m^3/J

In [ ]:
kT = statistics.mean(kT_block[#x:#y]) #fill with the proper x,y
dkT = statistics.mean(dkT_block[#x:#y])
print('isothermal compressibility=', kT, '+/-', dkT, '$Pa^{-1}$')

#### Follow the same procedure for calculating the 4th-order Binder ($U_L$) cumulant and the average density ($\langle \rho \rangle$).

### Compute intermediate scattering functions $C(k,t)$ of density and polarization fluctuations.

In [ ]:
L=np.mean(Lbox) #averaged box size

In [ ]:
nk = np.arange(0,20,1)

In [ ]:
#Create wavevector components (kx,ky,kz) along the three Cartesian axes: x, y, and z
k = nk*2*np.pi/L
kx = ky= kz = k

In [ ]:
Msm_av = np.zeros((totConf, 4, 1000))
Msm_avm = np.zeros((totConf, 4, 1000))
#
for comp in range(1, 4):
    Msm_av[:, comp, :] = np.mean(Msm[:, comp, :], axis=(0, 1), keepdims=True) # Molecular dipole averaged over all molecules and time
    Msm_avm[:, comp, :] = np.mean(Msm[:, comp, :], axis=0) # Single molecular dipole averaged over time 

In [ ]:
c_kt = np.zeros((k.size**3, totConf), dtype=complex) #charge density at wavevector (kx,ky,kz)
m0_kt = np.zeros((k.size**3, 3, totConf), dtype=complex) #dipole moment at wavevector (kx,ky,kz)
m_kt = np.zeros((k.size**3, 3, totConf), dtype=complex) #dipole moment fluctuations with respect to Msm_av at wavevector (kx,ky,kz)
mm_kt = np.zeros((k.size**3, 3, totConf), dtype=complex)  #dipole moment fluctuations with respect to Msm_avm at wavevector (kx,ky,kz)
m_ktLongm = np.zeros((k.size**3, totConf), dtype=complex) #projection of dipole moment fluctuations m_kt along the direction parallel to the total average polarization at wavevector (kx,ky,kz)
m_ktTram = np.zeros((k.size**3, 3, totConf), dtype=complex) #projection of dipole moment fluctuations m_kt in the direction orthogonal to the total average polarization at wavevector (kx,ky,kz) 
nCM_kt = np.zeros((k.size**3, totConf), dtype=complex) #particles number density at wavevector (kx,ky,kz)
m_ktTra = np.zeros((k.size**3, 3, totConf), dtype=complex) #dipole moment in the direction orthogonal to the average total polarization at wavevector (kx,ky,kz)
k_mod = np.zeros((k.size**3)) #magnitude of the wavevector (kx,ky,kz) 
#
def exp_sum(k_n, k_m, k_l, CM, Lbox, factor):
    return np.sum(np.exp(1j * (k_n * CM[:, 1, :] * L / Lbox[:, 1, :] +
                               k_m * CM[:, 2, :] * L / Lbox[:, 1, :] +
                               k_l * CM[:, 3, :] * L / Lbox[:, 1, :])) * factor, axis=1)
#
for n, k_n in enumerate(kx):
    for m, k_m in enumerate(ky):
        for l, k_l in enumerate(kz):
            idx = n * k.size**2 + m * k.size + l
            k_mod[idx] = np.sqrt(k_n**2 + k_m**2 + k_l**2)  # nm^-1
            c_kt[n*k.size**2+m*k.size+l,:] = np.sum(np.exp(1j*(kx[n]*coo[:,0,:]+ky[m]*coo[:,1,:]+kz[l]*coo[:,2,:]))*chargetotConf[:,:], axis = 1) # eV
            nCM_kt[idx, :] = exp_sum(k_n, k_m, k_l, CM, Lbox, 1)
            for comp in range(3):
                m0_kt[idx, comp, :] = exp_sum(k_n, k_m, k_l, CM, Lbox, Msm[:, comp+1, :])
                factor_1 = Msm[:, comp + 1, :] - Msm_av[:, comp + 1, :]
                m_kt[idx, comp, :] = exp_sum(k_n, k_m, k_l, CM, Lbox, factor_1)
                factor_2 = Msm[:, comp + 1, :] - Msm_avm[:, comp + 1, :]
                mm_kt[idx, comp, :] = exp_sum(k_n, k_m, k_l, CM, Lbox, factor_2)
            #
            m_ktLongm[idx, :] = np.sum(m_kt[idx, i, :] * (np.mean(M[:,i+1]) / np.mean(M[:,4])) for i in range(3))    
            #
            k_vec = np.array([k_n, k_m, k_l]) / k_mod[idx]
            m_ktTra[idx, :, :] = np.cross(k_vec[:, np.newaxis], m0_kt[idx, :, :], axis=0)
            #
            M_mean_vec = np.mean(M[:, 1:4], axis=0) / np.mean(M[:, 4])
            m_ktTram[idx, :, :] = np.cross(M_mean_vec[:, np.newaxis], m_kt[idx, :, :], axis=0)

In [ ]:
delta_k=np.zeros((k_mod.size,totConf))
delta_k[0,:]=1
nCM_kt=nCM_kt-1000*delta_k

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(k_mod,'+')
plt.xlabel("n_sort", fontsize=20)
plt.ylabel("|$\\bf{k}$|[$nm^{-1}$]", fontsize=20)
plt.title('k_mod ', fontdict=None, loc='center', pad=None)
plt.show()

In [ ]:
k_mod_eq=np.zeros((k_mod.size,k_mod.size))

In [ ]:
#Identifying wavectors with the same magnitude within a tollerance of 1.8
for i in range (0,k_mod.size):
    modk=k_mod[i]
    for n in range (0,k_mod.size):
        if modk-1.8<k_mod[n]<modk+1.8:
            k_mod_eq[i,n]=1

In [ ]:
t = M[:,0] # time axis in ps

In [ ]:
# Initialize correlation arrays
corr_vars = ['corr_k', 'corr_kNorm', 'corr_raw', 'corr_nCMk', 'corr_nCMkNorm', 'corr_nCMraw', 
             'corr_Longm', 'corr_LongmNorm', 'corr_Longmraw', 'corr_Tram', 'corr_TramNorm', 
             'corr_Tramraw', 'corr_mm', 'corr_mmNorm', 'corr_mmraw']

corr = {var: np.zeros((k_mod.size, totConf if 'raw' not in var else 2*totConf - 1), dtype=np.complex_) for var in corr_vars}

# calculate correlation functions at each wavevectors (kx,ky,kz) 
for j in range(k_mod.size):
    # nCM correlation 
    corr['corr_nCMraw'][j, :] = signal.correlate(nCM_kt[j, :], nCM_kt[j, :], mode='full', method='fft')
    corr['corr_nCMk'][j, :] = corr['corr_nCMraw'][j, totConf-1:] / (totConf - np.arange(totConf))
    
    # m_ktLongm correlation
    corr['corr_Longmraw'][j, :] = signal.correlate(m_ktLongm[j, :], m_ktLongm[j, :], mode='full', method='fft')
    corr['corr_Longm'][j, :] = corr['corr_Longmraw'][j, totConf-1:] / (totConf - np.arange(totConf))
    
    # m_ktTram correlation 
    for axis in range(3):
        corr['corr_Tramraw'][j, :] += signal.correlate(m_ktTram[j, axis, :], m_ktTram[j, axis, :], mode='full', method='fft')
    corr['corr_Tram'][j, :] = corr['corr_Tramraw'][j, totConf-1:] / (totConf - np.arange(totConf))
    
    # mm correlation 
    for axis in range(3):
        corr['corr_mmraw'][j, :] += signal.correlate(mm_kt[j, axis, :], mm_kt[j, axis, :], mode='full', method='fft')
    corr['corr_mm'][j, :] = corr['corr_mmraw'][j, totConf-1:] / (totConf - np.arange(totConf))

In [ ]:
# Initialize averaged correlation arrays
corr_averages = {name: np.zeros((k_mod.size, totConf)) for name in ['corr_kaverage', 'corr_nCMkaverage', 
                                                                    'corr_Longmaverage', 'corr_Tramaverage', 
                                                                    'corr_mmaverage']}

# Counting the number of wavectors with the same magnitude within the specified tollerance of 1.8
for i in range(k_mod.size):
    norm_factor = np.count_nonzero(k_mod_eq[1:, i])
    if norm_factor==0:
        norm_factor=1
    # Calculate averaged correlations over different wavector direction with the same magnitude within a tollerance of 1.8 
    corr_averages['corr_nCMkaverage'][i, :] = np.dot(k_mod_eq[1:, i], corr_nCMk[1:, :]).real / norm_factor
    corr_averages['corr_Longmaverage'][i, :] = np.dot(k_mod_eq[1:, i], corr_Longm[1:, :]).real / norm_factor
    corr_averages['corr_Tramaverage'][i, :] = np.dot(k_mod_eq[1:, i], corr_Tram[1:, :]).real / norm_factor
    corr_averages['corr_mmaverage'][i, :] = np.dot(k_mod_eq[1:, i], corr_mm[1:, :]).real / norm_factor


In [ ]:
# Calculate averaged k_mod values
k_modaverage = np.sum(k_mod[:, None] * k_mod_eq, axis=0) / np.count_nonzero(k_mod_eq, axis=0)

In [ ]:
# Initialize arrays in a compact form using a dictionary 
corr_averageOrd = {key: np.zeros((k_mod.size, totConf, 2)) for key in ['corr_kaverageOrd', 'corr_nCMkaverageOrd', 'corr_LongmaverageOrd', 'corr_TramaverageOrd', 'corr_mmaverageOrd']}

In [ ]:
# Store the arrays in a dictionary 
corr_dict = {
    'corr_kaverageOrd': corr_kaverageOrd,
    'corr_nCMkaverageOrd': corr_nCMkaverageOrd,
    'corr_LongmaverageOrd': corr_LongmaverageOrd,
    'corr_TramaverageOrd': corr_TramaverageOrd,
    'corr_mmaverageOrd': corr_mmaverageOrd
}

# Assign k_modaverage to the first column of each average array
for i in range(totConf):
    for key in corr_dict:
        corr_dict[key][:, i, 0] = k_modaverage

# Assign the average values to the second column of each average array
for key, values in zip(corr_dict.keys(), [corr_kaverage, corr_nCMkaverage, corr_Longmaverage, corr_Tramaverage, corr_mmaverage]):
    corr_dict[key][:, :, 1] = values

In [ ]:
# List of correlation average arrays
corr_arrays = [
    'corr_kaverageOrd',
    'corr_nCMkaverageOrd',
    'corr_LongmaverageOrd',
    'corr_TramaverageOrd',
    'corr_mmaverageOrd'
]

# Use a loop to apply np.unique to each array
for corr in corr_arrays:
    globals()[corr] = np.unique(globals()[corr], axis=0)

### Compute $\epsilon_{L(T)}(k)$, $S(k)$, $\langle \delta n \delta P_{L\hat{p}} \rangle(k)$, $\langle \delta P_{L\hat{p}}\delta P_{T\hat{p}}^2 \rangle (k)$.

In [ ]:
factor = (10 * eAng_D)**2 / (kB * T * e_0 * V_mean * 3)  #pre-factors for dimensionless susceptibility
factor_cross = (10 * eAng_D)**6 / ((kB * T)**3 * e_0**3 * V_mean**3 * 3**3)  
chi_Longk = factor * np.var(c_kt, axis=1) / k_mod**2 #Non-local susceptibility longitudinal to the wavevector (kx,ky,kz)
chi_Tramk = factor * np.sum([np.var(m_ktTra[:, i, :], axis=1) for i in range(3)], axis=0) #Non-local susceptibility orthogonal to the wavevector (kx,ky,kz)
chi_nCMk = np.mean(np.abs(nCM_kt)**2, axis=1) / 1000  #Static structure factor (center of mass)
chi_Longmmk = factor * np.mean(np.abs(m_ktLongm)**2, axis=1) #Non-local susceptibility longitudinal to the average total polarization vector
chi_Trammk = factor * np.sum([np.mean(np.abs(m_ktTram[:, i, :])**2, axis=1) for i in range(3)], axis=0) #Non-local susceptibility orthogonal to the average total polarization vector
chi_nCMLongmmk = factor * np.mean(np.abs(m_ktLongm * np.conj(nCM_kt))**2, axis=1) / 1000 #Static correlation between density fluctuations and polarization fluctuations longitudinal to the average total polarization vector
chi_LongTrammk = factor_cross * np.mean(np.abs(np.sum(m_ktTram**2, axis=1) * np.conj(m_ktLongm))**2, axis=1) #Static correlation between longitudinal polarization fluctuations and the square of the fluctuations orthogonal to the average polarization vector

In [ ]:
chi_vars = [
    'chi_Longk', 'chi_nCMk', 'chi_Tramk', 'chi_Longmmk', 
    'chi_Trammk', 'chi_mmk', 'chi_nCMLongmmk', 'chi_LongTrammk', 'k_mod'
]
#
# Initialize arrays to store averages for each variable
chi_avgs = {var: np.zeros(nCM_kt[:, 0].size) for var in chi_vars}

#Averaging along different directions of wavevectors with the same magnitude, with a tolerance of 1.8
for i in range(nCM_kt[:, 0].size):
    nonzero_count = np.count_nonzero(k_mod_eq[1:, i]) 
    if nonzero_count==0:
        nonzero_count=1
    for var in chi_vars:
        chi_avgs[var][i] = np.sum(eval(var)[1:] * k_mod_eq[1:, i]) / nonzero_count

In [ ]:
ordered_averages = {
    'chi_LongkaverageOrd': np.zeros((chi_Longkaverage.size, 2)),
    'chi_nCMkaverageOrd': np.zeros((chi_Longmmkaverage.size, 2)),
    'chi_TramkaverageOrd': np.zeros((chi_Longmmkaverage.size, 2)),
    'chi_LongmmkaverageOrd': np.zeros((chi_Longmmkaverage.size, 2)),
    'chi_TrammkaverageOrd': np.zeros((chi_Trammkaverage.size, 2)),
    'chi_nCMLongmmkaverageOrd': np.zeros((chi_nCMLongmmkaverage.size, 2)),
    'chi_LongTrammkaverageOrd': np.zeros((chi_LongTrammkaverage.size, 2))
}

In [ ]:
average_pairs = [
    ('chi_LongkaverageOrd', chi_Longkaverage),
    ('chi_nCMkaverageOrd', chi_nCMkaverage),
    ('chi_TramkaverageOrd', chi_Tramkaverage),
    ('chi_LongmmkaverageOrd', chi_Longmmkaverage),
    ('chi_TrammkaverageOrd', chi_Trammkaverage),
    ('chi_nCMLongmmkaverageOrd', chi_nCMLongmmkaverage),
    ('chi_LongTrammkaverageOrd', chi_LongTrammkaverage)
]

for var_name, avg_array in average_pairs:
    ordered_averages[var_name][:, 0] = k_modaverage  
    ordered_averages[var_name][:, 1] = avg_array     

In [ ]:
ordered_averages = [
    chi_LongkaverageOrd, chi_nCMkaverageOrd, chi_TramkaverageOrd, 
    chi_LongmmkaverageOrd, chi_TrammkaverageOrd, 
    chi_nCMLongmmkaverageOrd, chi_LongTrammkaverageOrd
]

#Remove duplicates and sort by increasing values of the magnitude of the wavevector
ordered_averages = [np.unique(arr, axis=0) for arr in ordered_averages]

In [ ]:
epsilon_Longk=1/(1-chi_LongkaverageOrd[1:,1]) #static dielectric function longitudinal to the wavevector (kx,ky,kz)
epsilon_Tramk=1+chi_TramkaverageOrd[1:,1] #static dielectric function orthogonal to the wavevector (kx,ky,kz)